# AI back test methods1

# --2 fonctions and dependancys


In [ ]:
#! python3 -m pip install --upgrade pip
#! pip install mplfinance
from xdata_config import *
from functions_module import *
import mplfinance as mpf
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import MaxNorm
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense 
from keras.models import load_model
from datetime import datetime

In [ ]:
import requests

url = 'https://api.binance.com/api/v3/ticker/price'

response = requests.get(url)
tickers = response.json()
ticker_list=[]
for ticker in tickers:
    ticker_list.append((ticker['symbol']))


# --1 Config

In [147]:
# ai_options
Normalization_File= "False"
Model_FileName= "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re4.h5"
WINDOW_SIZE= 7
PRESSISION=0.0
PERIODE_START="2023-02-09 00:00:00"
PERIODE_END="2023-02-11 00:00:00"

# start_period = datetime.strptime(PERIODE_START, '%d-%m-%Y %H:%M:%S')
# end_period = datetime.strptime(PERIODE_END, '%d-%m-%Y %H:%M:%S')
start_period = pd.Timestamp(PERIODE_START)
end_period = pd.Timestamp(PERIODE_END)


#trading_options:
TAKE_PROFIT= 0.4  
STOP_LOSS=  0.8 
MAX_HOLDING_TIME= 16

USE_TRAILING_STOP_LOSS: False
TRAILING_STOP_LOSS= .002     
TRAILING_TAKE_PROFIT= .002  

PAIR_WITH= "USDT"
TRADE_TOTAL= 100 
TRADE_SLOTS= 5

TRADING_FEE= 0.1
  

# --3 Importing DATA

In [ ]:
from datetime import date
#TICKERS = "../Binance-Fast-Trade-Bot/volatile_volume_" + str(date.today()) + ".txt"
TICKERS = "/UltimeTradingBot/Binance-Fast-Trade-Bot/halal_coins.txt"
VOLATILE_COINS=[line.strip() for line in open(TICKERS)]
PAIR_WITH="USDT"
VOLATILE_USDT_PAIRS=[coin+"/USDT" for coin in VOLATILE_COINS]
VOLATILE_BUSD_PAIRS=[coin+"/BUSD" for coin in VOLATILE_COINS]
for pair in VOLATILE_BUSD_PAIRS:
    if pair.replace('/' ,'') not  in ticker_list:
        print(pair)
        VOLATILE_BUSD_PAIRS.remove(pair)

for pair in VOLATILE_USDT_PAIRS:
    if pair.replace('/', '') not in ticker_list:
        print(pair)
        VOLATILE_USDT_PAIRS.remove(pair)

content = os.listdir('database/DataBackTest/1m')
for pair in VOLATILE_USDT_PAIRS:
    if pair.replace('/', '-')+'.csv' not in content:
        print(pair)
        VOLATILE_USDT_PAIRS.remove(pair)
for pair in VOLATILE_BUSD_PAIRS:
    if pair.replace('/', '-')+'.csv' not in content:
        print(pair)
        VOLATILE_BUSD_PAIRS.remove(pair)

In [ ]:
# coins_to_download=''
# for coin in LISTCOIN:
#     coins_to_download=coins_to_download+" "+coin
# os.system(f"node database/dd_for_backtest.js {coins_to_download} {PAIR_WITH}")#node database/ddargs.js ORN BUSD

In [ ]:
pair_list = VOLATILE_USDT_PAIRS
#tf = '1m'
oldest_pair = "BTC/USDT"
if oldest_pair not in pair_list: pair_list.append(oldest_pair)
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")



# --3 Decalring Tables

In [156]:
import pandas as pd

def create_portfolio_dataframe(num_slots,init_quantity=500, base_currency='usdt'):
    # create an empty DataFrame with the columns we need
    columns = ['date', f'reserve_{base_currency}',f'total_{base_currency}']
    for i in range(num_slots):
        columns += [f'slot{i+1}_symbol', f'slot{i+1}_volume', f'slot{i+1}_original_price_{base_currency}', f'slot{i+1}_current_total_{base_currency}']
    df = pd.DataFrame(columns=columns)
    
    # create some sample data
    for date in pd.date_range(PERIODE_START, periods=1):
        data = {'date': date, f'total_{base_currency}': init_quantity}
        # for i in range(num_slots):
        #     symbol = f'coin{i+1}'
        #     volume = (i+1) * 100
        #     bought_currency = base_currency.upper()
        #     bought_value = (i+1) * 500
        #     current_price = (i+1) * 1.5
        #     current_total = volume * current_price
        #     data.update({
        #         f'slot{i+1}_symbol': symbol,
        #         f'slot{i+1}_volume': volume,
        #         f'slot{i+1}_original_price_{base_currency}': bought_value,
        #         f'slot{i+1}_current_total_{base_currency}': current_total,
        #     })
        #     data[f'total_{base_currency}'] += current_total
        df = df.append(data, ignore_index=True)
    
    # set the date column as the index of the DataFrame
    df.set_index('date', inplace=True)
    df.iloc[:,0]=np.float64(init_quantity)
    return df.iloc[0:1]



def generate_signals(AllTop20Volumes, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, buy_test, backtest_model):
    signal_df = pd.DataFrame(columns=['coin', 'time', 'price', 'note'])
    for day, TOPLIST in AllTop20Volumes.items():
        for coin in TOPLIST:
            try:
                print(f">>>>>>>>>>> working on {coin} at: {day} :")
                loc_start = df_list1m[coin].index.get_loc(day)
                loc_end = df_list1m[coin].index.get_loc(day+pd.Timedelta('1 day'))
                gc.collect()
                df = mini_expand4(pair=coin, i=loc_start, j=loc_end, window=WINDOW_SIZE, metadata=MetaData, buy_pourcent=TAKE_PROFIT, sell_pourcent=STOP_LOSS, buy_function=buy_alwase)
                dt = df.iloc[:,:-1].to_numpy(dtype=np.float32)
                predictions_note = backtest_model.predict(dt)
                predictions_round = predictions_note.round()
                dico_signal = {"coin":coin, "time":df[predictions_round==1].index.values, "price":df[predictions_round==1]["price"].values, "note":predictions_note[predictions_round==1]}
                df_signal_coin = pd.DataFrame(dico_signal)
                signal_df = pd.concat([signal_df, df_signal_coin])
            except:
                print(f"error at {day} in {coin}")
    return signal_df
#signal_df = generate_signals(AllTop20Volumes, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, buy_test, backtest_model)

In [157]:
Portfolio=create_portfolio_dataframe(TRADE_SLOTS,init_quantity=500, base_currency=PAIR_WITH) #pair with plus slots plus total
Order_history= pd.DataFrame(columns= ['Order ID', 'Pair', 'Side', 'Price', 'Quantity', 'Executed', 'Time', 'Status'])
signal_df=pd.DataFrame(columns= ['coin', 'time',"price",'note'])

In [ ]:
Portfolio.columns

In [155]:

def get_top_volumes(start_period,end_period):
    AllTop20Volumes={}
    for day in df_list1d["BTC/USDT"].index:
        if start_period<= day <= end_period:
            Top20vol={}
            for p,df in df_list1d.items():
                if p not in ["EUR/USDT","EUR/BUSD","BTC/USDT","BTC/BUSD"]:
                    try:Top20vol.update({p:(df.loc[day].volume*df.loc[day].close)})
                    except Exception as e :print(f'Time Error wile working on {p}: {e}')
                    #print (Top20vol)
            AllTop20Volumes.update({day:sorted(Top20vol, key=Top20vol.get, reverse=True)[:20]})
    return AllTop20Volumes

def is_coin_in_portfolio(symbol,instant,Portfolio):
    for i in range(1,TRADE_SLOTS+1):
        try:
            if (Portfolio[f"slot{i}_symbol"].loc[instant]==symbol):
                return i
        except Exception as e:
            print(f"error :{e}")
    return False

In [ ]:
AllTop20Volumes=get_top_volumes(start_period,end_period)
AllTop20Volumes[pd.Timestamp("2023-02-09")]

In [ ]:

backtest_model=load_model("/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_VeryDeep.h5")

In [ ]:

signal_df=pd.DataFrame(columns= ['coin', 'time',"price",'note'])
for day,TOPLIST in AllTop20Volumes.items():
    for coin in TOPLIST:
        try:
            print(f">>>>>>>>>>> working on {coin} at: {day} :")
            loc_start=df_list1m[coin].index.get_loc(day)
            loc_end=df_list1m[coin].index.get_loc(day+pd.Timedelta('1 day'))
            gc.collect()
            df=mini_expand4(pair=coin,i=loc_start,j=loc_end,window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=TAKE_PROFIT,sell_pourcent=STOP_LOSS,buy_function=buy_alwase)
            dt=df.iloc[:,:-1].to_numpy(dtype=np.float32)
            predictions_note=backtest_model.predict(dt)
            predictions_round=predictions_note.round()
            dico_signal={"coin":coin,"time":df[predictions_round==1].index.values,"price":df[predictions_round==1]["price"].values,"note":predictions_note[predictions_round==1]}
            df_signal_coin = pd.DataFrame(dico_signal)
            signal_df=pd.concat([signal_df,df_signal_coin])
        except:
            print(f"error at {day} in {coin}")

In [ ]:
signal_df = pd.DataFrame(columns=['coin', 'time', 'price', 'note'])

for day, TOPLIST in AllTop20Volumes.items():
    for coin in TOPLIST:
        try:
            print(f">>>>>>>>>>> working on {coin} at: {day} :")
            loc_start = df_list1m[coin].index.get_loc(day)
            loc_end = df_list1m[coin].index.get_loc(day + pd.Timedelta('1 day'))
            gc.collect()

            # Get the required columns from the dataframe
            df = mini_expand5(pair=coin, i=loc_start, j=loc_end, window=WINDOW_SIZE, metadata=MetaData,
                              buy_pourcent=TAKE_PROFIT, sell_pourcent=STOP_LOSS, buy_function=buy_alwase)
            # Add the prediction note to the dataframe
            df.pop("buy")
            dt = df.to_numpy(dtype=np.float32)
            predictions_note = backtest_model.predict(dt)
            df["time"] = df.index.values
            predictions_round = predictions_note.round()
            df['note'] = predictions_note

            # Filter the rows with positive predictions and create a dataframe with the signal
            df_signal_coin = df[predictions_round == 1][['time', 'price', 'note']].reset_index(drop=True)
            df_signal_coin['coin'] = coin

            signal_df = pd.concat([signal_df, df_signal_coin])
        except:
            print(f"error at {day} in {coin}")

In [93]:
signal_df

,coin,time,price,note
0,XRP/USDT,2023-02-09 02:59:00,0.395000,0.510234
1,XRP/USDT,2023-02-09 03:00:00,0.394325,0.729723
2,XRP/USDT,2023-02-09 03:01:00,0.393150,0.825384
3,XRP/USDT,2023-02-09 03:02:00,0.392450,0.837969
4,XRP/USDT,2023-02-09 03:03:00,0.392375,0.671343
...,...,...,...,...
497,AR/USDT,2023-02-11 23:16:00,11.685000,0.509952
498,AR/USDT,2023-02-11 23:17:00,11.685000,0.522397
499,AR/USDT,2023-02-11 23:18:00,11.675000,0.534739
500,AR/USDT,2023-02-11 23:19:00,11.677500,0.516791


In [ ]:
coin="XRP/USDT"
day=pd.Timestamp("2023-02-10 00:00:00")
loc_start=df_list1m[coin].index.get_loc(day)
loc_end=df_list1m[coin].index.get_loc(day+pd.Timedelta('1 day'))
gc.collect()
df=mini_expand4(pair=coin,i=loc_start,j=loc_end,window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=TAKE_PROFIT,sell_pourcent=STOP_LOSS,buy_function=buy_alwase)
df.pop("buy")
dt = df.to_numpy(dtype=np.float32)
predictions_note = backtest_model.predict(dt)
df["time"] = df.index.values
predictions_round = predictions_note.round()
df['note'] = predictions_note

# Filter the rows with positive predictions and create a dataframe with the signal
df_signal_coin = df[predictions_round == 1][['time', 'price', 'note']].reset_index(drop=True)
df_signal_coin['coin'] = coin

signal_df = pd.concat([signal_df, df_signal_coin])
# dt=df.iloc[:,:-1].to_numpy(dtype=np.float32)
# predictions_note=backtest_model.predict(dt)
# predictions_round=predictions_note.round()
# dico_signal={"coin":coin,"time":df[predictions_round==1].index.values,"price":df[predictions_round==1]["price"].values,"note":predictions_note[predictions_round==1]}
# df_signal_coin = pd.DataFrame(dico_signal)
#signal_df=pd.concat([signal_df,df_signal_coin])

In [ ]:
df_list1m[coin].index.get_loc(day+pd.Timedelta('1 day'))

In [ ]:
def mini_expand5(pair="GMT/USDT", i=0, j=10000, window=2, metadata=MetaData,
                 high_weight=1, buy_pourcent=BUY_PERCENT, sell_pourcent=SELL_PERCENT,
                 buy_function=buy_min_up):
    print(f"mini_expand : {pair}")
    pair_df = df_list1m[pair].iloc[i:j]
    btc_df = df_list1m["BTC/USDT"].loc[(pair_df.index[0] - pd.DateOffset(days=window+1)).round(freq='1 min'):pair_df.index[-1]+pd.Timedelta(f"{window} day")]
    Pair_Full = full_expand(pair_df, df_list5m[pair], df_list15m[pair], df_list1h[pair], df_list1d[pair], window)
    BTC_Full = full_expand(btc_df, df_list5m["BTC/USDT"], df_list15m["BTC/USDT"], df_list1h["BTC/USDT"], df_list1d["BTC/USDT"], window)   
    BTC_Full = BTC_Full.add_prefix("BTC_")
    Merged = pd.merge(Pair_Full, BTC_Full, left_index=True, right_index=True)
    day_expand(Merged)
    Meta_expand(Merged, metadata, pair)
    Merged=buy_function(Merged, buy_pourcent=buy_pourcent, sell_pourcent=sell_pourcent, window=MAX_FORCAST_SIZE)
    Merged["high"] = (Merged["open"] + high_weight * Merged["high"] + Merged["low"] + Merged["close"]) / (3 + high_weight)
    Merged["BTC_high"] = (Merged["BTC_open"] + high_weight * Merged["BTC_high"] + Merged["BTC_low"] + Merged["BTC_close"]) / (3 + high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    # Arr_prices=Merged.columns.str.contains("open") | Merged.columns.str.contains("high") |Merged.columns.str.contains("low")| Merged.columns.str.contains("close")
    # BTC_Arr_prices= Merged.columns.str.contains("BTC") & Arr_prices
    # Coin_Arr_prices= ~Merged.columns.str.contains("BTC") & Arr_prices
    # Merged.loc[:,  BTC_Arr_prices ] = 1-(Merged.loc[:, BTC_Arr_prices]/ Merged["BTC_price"])
    # Merged.loc[:,Coin_Arr_prices] = 1-(Merged.loc[:,Coin_Arr_prices] / Merged["price"])
    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    Merged=Merged.dropna()
    print(f'######################  mini_expand5 {pair} - shape {Merged.shape}  buy mean : {hp(Merged.buy.mean())} ############################')
    return Merged


In [ ]:
# df=mini_expand4(pair=coin,i=loc_start,j=loc_end,window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=TAKE_PROFIT,sell_pourcent=STOP_LOSS,buy_function=buy_alwase)
df

In [ ]:
df2=mini_expand5(pair=coin,i=loc_start,j=loc_end,window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=TAKE_PROFIT,sell_pourcent=STOP_LOSS,buy_function=buy_alwase)
df2

In [99]:

def backtest_buy_sell(Portfolio, Order_history, signal_df, df_list1m, start_period, end_period):
    # Define some constants
    BUY_SIDE = 'BUY'
    SELL_SIDE = 'SELL'
    EXECUTED_STATUS = 'EXECUTED'
    CANCELLED_STATUS = 'CANCELLED'

    # Loop through all the rows in the signal dataframe
    for index, row in signal_df.iterrows():
        coin = row['coin']
        time = row['time']
        price = row['price'] # We assume there's only one price per signal
        note = row['note'] # We assume there's only one note per signal

        # Check if the coin is already in the portfolio
        slot_num = is_coin_in_portfolio(coin, time)
        if slot_num:
            # If the coin is already in the portfolio, update the existing slot
            slot_symbol = f'slot{slot_num}_symbol'
            slot_volume = f'slot{slot_num}_volume'
            slot_original_price_usdt = f'slot{slot_num}_original_price_USDT'
            slot_current_total_usdt = f'slot{slot_num}_current_total_USDT'

            # Calculate the new volume and bought USDT based on the current price
            current_total_usdt = Portfolio['total_USDT'].iloc[-1]
            new_volume = (Portfolio[slot_original_price_usdt].iloc[-1] * (1 - TRADING_FEE)) / price
            new_original_price_usdt = new_volume * price

            # Update the portfolio dataframe
            Portfolio.at[time, slot_volume] = new_volume
            Portfolio.at[time, slot_original_price_usdt] = new_original_price_usdt
            Portfolio.at[time, slot_current_total_usdt] = new_original_price_usdt + (current_total_usdt - Portfolio[slot_original_price_usdt].iloc[-1])

            # Add an order to the order history dataframe
            order_id = len(Order_history) + 1
            order_side = SELL_SIDE if note < 0 else BUY_SIDE
            order_executed = abs(note)
            order_quantity = new_volume
            order_status = EXECUTED_STATUS
            order_price = price
            order_time = time
            order_pair = f'{coin}/{PAIR_WITH}'

            Order_history = Order_history.append({'Order ID': order_id, 'Pair': order_pair, 'Side': order_side,
                                                  'Price': order_price, 'Quantity': order_quantity,
                                                  'Executed': order_executed, 'Time': order_time,
                                                  'Status': order_status}, ignore_index=True)
        else:
            # If the coin is not in the portfolio, find the first empty slot
            for i in range(1, TRADE_SLOTS + 1):
                if pd.isna(Portfolio[f'slot{i}_symbol'].iloc[-1]):
                    # Calculate the volume and bought USDT based on the current price
                    volume = (MAX_TRADE_PER_SLOTE * (1 - TRADING_FEE)) / price
                    bought_usdt = volume * price

                    # Update the portfolio dataframe
                    Portfolio.at[time, f'slot{i}_symbol'] = coin
                    Portfolio.at[time, f'slot{i}_volume'] = volume
                    Portfolio.at[time, f'slot{i}_original_price_USDT'] = bought_usdt
                    Portfolio.at[time, f'slot{i}_current_total_USDT'] = bought_usdt + Portfolio['total_USDT'].iloc[-1]
                # Add an order to the order history dataframe
                order_id = len(Order_history) + 1
                order_side = BUY_SIDE
                order_executed = abs(note)
                order_quantity = volume
                order_status = EXECUTED_STATUS
                order_price = price
                order_time = time
                order_pair = f'{coin}/{PAIR_WITH}'

                Order_history = Order_history.append({'Order ID': order_id, 'Pair': order_pair, 'Side': order_side,
                                                        'Price': order_price, 'Quantity': order_quantity,
                                                        'Executed': order_executed, 'Time': order_time,
                                                        'Status': order_status}, ignore_index=True)

                break

    return Portfolio, Order_history
                    


In [158]:
Portfolio.columns

Index(['reserve_USDT', 'total_USDT', 'slot1_symbol', 'slot1_volume',
       'slot1_original_price_USDT', 'slot1_current_total_USDT', 'slot2_symbol',
       'slot2_volume', 'slot2_original_price_USDT', 'slot2_current_total_USDT',
       'slot3_symbol', 'slot3_volume', 'slot3_original_price_USDT',
       'slot3_current_total_USDT', 'slot4_symbol', 'slot4_volume',
       'slot4_original_price_USDT', 'slot4_current_total_USDT', 'slot5_symbol',
       'slot5_volume', 'slot5_original_price_USDT',
       'slot5_current_total_USDT'],
      dtype='object')

In [ ]:
is_coin_in_portfolio(coin, pd.Timestamp() )

The code is a backtesting strategy to simulate buying and selling cryptocurrencies using a given signal DataFrame and historical prices of the cryptocurrencies.

The Portfolio DataFrame is used to keep track of the state of the portfolio at each point in time during the backtesting. It contains information about the reserved and total amounts of the base currency (USDT), as well as the symbols, volumes, original prices, and current total values of up to 5 cryptocurrencies held in slots.

The fisrt_empty_slot() function checks if there is an empty slot in the portfolio where a new cryptocurrency can be added. If there is an empty slot, it returns the slot number. Otherwise, it returns False.

The buy_coin() function simulates buying a cryptocurrency at a given time and price. It checks if the cryptocurrency is not already in the portfolio and if there is enough reserved base currency to buy it. If the conditions are met, it updates the Portfolio DataFrame by subtracting the base currency used for buying and adding the bought cryptocurrency to an empty slot. It also prints a message indicating the amount and symbol of the bought cryptocurrency. Otherwise, it prints a message indicating that the buying cannot be performed.

The upadate_slots() function updates the Portfolio DataFrame for the current time by computing the current values of all the cryptocurrencies held in the slots using the historical prices of the cryptocurrencies. It then updates the slot information in the Portfolio DataFrame.

The stop_loss_or_take_profit() function is not implemented in the provided code.

The backtest_buy_sell() function is the main function that performs the backtesting. It initializes the Portfolio and Order_history DataFrames, defines some constants, and loops over the time range specified by the start_period and end_period parameters. At each time step, it checks the signal DataFrame for a buying or selling signal, and if there is a buying signal, it calls the buy_coin() function. If there is a selling signal, it calls the stop_loss_or_take_profit() function (not implemented) to check if the stop-loss or take-profit conditions are met, and if they are, it simulates selling the cryptocurrency by calling the sell_coin() function (not implemented). It then updates the Portfolio and Order_history DataFrames for the current time step by calling the upadate_slots() function. Finally, it checks if the maximum holding time for each slot has been exceeded, and if it has, it simulates selling the cryptocurrency in that slot by calling the sell_coin() function (not implemented). The function returns the Portfolio and Order_history DataFrames at the end of the backtesting.

In [150]:
def fisrt_empty_slot(time,Portfolio=Portfolio):
    for i in range(1,TRADE_SLOTS+1):
        if pd.isna(Portfolio[f'slot{i}_symbol'].iloc[-1]):
            return i
    return False

def buy_coin(time,coin,price,Portfolio=Portfolio,pair_whith_qte=TRADE_TOTAL):
    previous_minute=time-pd.Timedelta("1 minute")
    slot_num = is_coin_in_portfolio(coin, previous_minute, Portfolio)
    previous_reserve=Portfolio.loc[previous_minute,f'reserve_{PAIR_WITH}']
    slot_ft=fisrt_empty_slot(previous_minute,Portfolio)
    if (slot_num==False) and (previous_reserve >= pair_whith_qte ) :
        Portfolio.at[time,f'reserve_{PAIR_WITH}']=previous_reserve-previous_reserve
        Portfolio.at[time,f'total_{PAIR_WITH}']=Portfolio.at[previous_minute,f'total_{PAIR_WITH}']-TRADING_FEE*pair_whith_qte/100
        Portfolio.at[time,f'slot{slot_ft}_symbol']=coin
        Portfolio.at[time,f'slot{slot_ft}_volume']=price*(pair_whith_qte-(TRADING_FEE*pair_whith_qte/100))
        Portfolio.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}']=price
        Portfolio.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}']=price*Portfolio.at[time,f'slot{slot_ft}_volume']
        print(f"good condition for buying {Portfolio.at[time,f'slot{slot_ft}_volume']} x {coin} at {price} ")
    else:
        print('xxx we cannot buy  {coin} at {price}  in {time} xxx')
    return Portfolio

def upadate_slots(time,df_list1m,Portfolio=Portfolio):
    previous_minute=time-pd.Timedelta("1 minute")
    previous_reserve=Portfolio.at[previous_minute,f'reserve_{PAIR_WITH}']
    Portfolio.at[time,f'reserve_{PAIR_WITH}']=previous_reserve
    previous_total=Portfolio.at[previous_minute,f'total_{PAIR_WITH}']
    for slot_ft in range(1,TRADE_SLOTS+1):
        if not pd.isna(Portfolio.at[previous_minute,f'slot{slot_ft}_symbol']):
            #get previous data
            coin=Portfolio.at[previous_minute,f'slot{slot_ft}_symbol']
            bought_at=Portfolio.at[previous_minute, f'slot{slot_ft}_original_price_{PAIR_WITH}']
            previous_total_slot_value=Portfolio.at[previous_minute, f'slot{slot_ft}_current_total_{PAIR_WITH}']
            slot_volume=Portfolio.at[previous_minute,f'slot{slot_ft}_volume']
            #Computation 
            new_total_slot_value=df_list1m[coin].at["","close"]*slot_volume
            slot_value_difference=new_total_slot_value-previous_total_slot_value
            new_total_assets_value=previous_total+slot_value_difference
            #update the slot using new price form df_list1m dataset
            Portfolio.at[time,f'slot{slot_ft}_symbol']=coin
            Portfolio.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}']=bought_at
            Portfolio.at[time,f'slot{slot_ft}_volume']=slot_volume
            Portfolio.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}']=new_total_slot_value
            Portfolio.at[time,f'total_{PAIR_WITH}']=new_total_assets_value           
    return Portfolio
  
def stop_loss_or_take_profit(time, df_list1m, Portfolio, st_pct, tp_pct, trade_fee):
    for slot_ft in range(1, TRADE_SLOTS+1):
        coin=Portfolio.at[time, f"slot{slot_ft}_symbol"]
        if Portfolio.at[time, f"slot{slot_ft}_symbol"] == coin:
            # Check if stop loss or take profit order should be triggered
            bought_price = Portfolio.at[time, f"slot{slot_ft}_original_price_{PAIR_WITH}"]
            slot_volume = Portfolio.at[time, f"slot{slot_ft}_volume"]
            current_price= df_list1m[coin].at(time,"close")
            slot_total_value = current_price * slot_volume
            slot_current_profit = ((current_price - bought_price) * slot_volume) - (trade_fee/100 * slot_total_value)
            slot_current_profit_pct = (slot_current_profit / (bought_price * slot_volume)) * 100

            if slot_current_profit_pct < st_pct:
                # Trigger stop loss
                Portfolio.at[time, f"slot{slot_ft}_symbol"] = np.nan
                Portfolio.at[time, f"slot{slot_ft}_volume"] = np.nan
                Portfolio.at[time, f"slot{slot_ft}_original_price_{PAIR_WITH}"] = np.nan
                Portfolio.at[time, f"slot{slot_ft}_current_total_{PAIR_WITH}"] = np.nan
                Portfolio.at[time, "reserve_USDT"] += slot_total_value
                print(f"Stop loss triggered for {coin} in slot {slot_ft}")
             #   return Portfolio

            elif slot_current_profit_pct > tp_pct:
                # Trigger take profit
                Portfolio.at[time, f"slot{slot_ft}_symbol"] = np.nan
                Portfolio.at[time, f"slot{slot_ft}_volume"] = np.nan
                Portfolio.at[time, f"slot{slot_ft}_original_price_{PAIR_WITH}"] = np.nan
                Portfolio.at[time, f"slot{slot_ft}_current_total_{PAIR_WITH}"] = np.nan
                Portfolio.at[time, "reserve_USDT"] += slot_total_value
                Portfolio.at[time, "total_USDT"] += (slot_total_value + (slot_current_profit_pct/100 * bought_price * slot_volume))
                print(f"Take profit triggered for {coin} in slot {slot_ft}")
                #return Portfolio
            
    return Portfolio

    
        
def backtest_buy_sell(signal_df, df_list1m, start_period, end_period):
    # Define some constants
    BUY_SIDE = 'BUY'
    SELL_SIDE = 'SELL'
    EXECUTED_STATUS = 'EXECUTED'
    CANCELLED_STATUS = 'CANCELLED'
    #initialize vaiable and counters
    curent_time=pd.Timestamp(start_period)
    end_period_marge=end_period+pd.Timedelta(f"{2*MAX_HOLDING_TIME} minute")
    Portfolio=create_portfolio_dataframe(TRADE_SLOTS,init_quantity=500, base_currency=PAIR_WITH) #pair with plus slots plus total
    Order_history= pd.DataFrame(columns= ['Order ID', 'Pair', 'Side', 'Price', 'Quantity', 'Executed', 'Time', 'Status'])
    PreiodRange = pd.date_range(start=pd.Timestamp(start_period), end=pd.Timestamp(end_period), freq='1min')
    # Verifie buying oportunities
    for curent_time in PreiodRange:
        previous_minute=curent_time-pd.Timedelta("1 minute")
        previous_reserve=Portfolio.loc(previous_minute,f'reserve_{PAIR_WITH}')
        upadate_slots(curent_time,df_list1m,Portfolio=Portfolio)
        print(curent_time)
        for i,sig_row in signal_df[signal_df.time==pd.Timestamp(curent_time)].iterrows():
            # buy the coin if solt and cash is avalible:
            coin = sig_row['coin']
            time = sig_row['time']
            price = sig_row['price'] # We assume there's only one price per signal
            note = sig_row['note'] 
            slot_num = is_coin_in_portfolio(coin, previous_minute, Portfolio)
            # buy first:
            if (note + PRESSISION) >= 0.5:
                Portfolio=buy_coin(time,coin,price,Portfolio=Portfolio,pair_whith_qte=TRADE_TOTAL)
        
        Portfolio=stop_loss_or_take_profit(curent_time, df_list1m, Portfolio, STOP_LOSS , TAKE_PROFIT, TRADING_FEE )
        
        ptc_benifit=100*Portfolio.at[curent_time,f'total_{PAIR_WITH}']/Portfolio[f'total_{PAIR_WITH}'].iloc[0]
        print(f"total benifit ptc {ptc_benifit}")       
    return Portfolio     
                
                
            
        
            

In [153]:
for i,row in signal_df[signal_df.time==pd.Timestamp("2023-02-10 23:59:00")].iterrows():
            # buy the coin if solt and cash is avalible:
            coin = row['coin']
            time = row['time']
            price = row['price'] # We assume there's only one price per signal
            note = row['note'] 
            print(price)

1.28775e-05
0.166525
0.0570275
3.191


In [151]:
backtest_buy_sell(signal_df, df_list1m, start_period, end_period)

2023-02-09 00:00:00
2023-02-09 00:01:00
2023-02-09 00:02:00
2023-02-09 00:03:00
2023-02-09 00:04:00
2023-02-09 00:05:00
2023-02-09 00:06:00
2023-02-09 00:07:00
0
0
0
0
0
2023-02-09 00:08:00
1
1
1
1
1
2023-02-09 00:09:00
2
0
2
0
2
2
2023-02-09 00:10:00
3
1
2
3
3
3
2023-02-09 00:11:00
4
2
3
4
4
4
2023-02-09 00:12:00
5
3
4
5
1
5
5
2023-02-09 00:13:00
6
4
5
6
2
6
6
2023-02-09 00:14:00
7
6
7
7
7
2023-02-09 00:15:00
8
7
8
8
8
2023-02-09 00:16:00
9
5
8
9
3
9
9
2023-02-09 00:17:00
10
6
9
10
4
10
10
2023-02-09 00:18:00
11
7
10
11
5
11
11
2023-02-09 00:19:00
12
8
11
12
6
12
12
2023-02-09 00:20:00
13
9
12
13
7
13
13
2023-02-09 00:21:00
14
10
13
14
8
14
14
2023-02-09 00:22:00
15
11
14
15
9
15
15
2023-02-09 00:23:00
0
16
12
15
16
0
10
16
16
2023-02-09 00:24:00
17
13
16
17
1
11
17
17
2023-02-09 00:25:00
18
14
17
18
2
12
0
18
18
2023-02-09 00:26:00
19
15
18
19
3
13
1
19
19
2023-02-09 00:27:00
20
16
19
20
4
14
20
20
2023-02-09 00:28:00
21
17
20
21
15
21
21
2023-02-09 00:29:00
22
18
21
22
16
22
22
2023

In [ ]:
MAX_TRADE_PER_SLOTE=100
Portfolio2,Order_history=backtest_buy_sell(Portfolio, Order_history, signal_df, df_list1m, start_period, end_period)

In [111]:
signal_df[signal_df.time==pd.Timestamp("2023-02-10 23:59:00")]

,coin,time,price,note
327,SHIB/USDT,2023-02-10 23:59:00,0.000013,0.581394
1291,GRT/USDT,2023-02-10 23:59:00,0.166525,0.593453
1136,SKL/USDT,2023-02-10 23:59:00,0.057028,0.650911
1277,HIGH/USDT,2023-02-10 23:59:00,3.191000,0.576563


In [ ]:
is_coin_in_portfolio(symbol="JASMY/USDT",instant="2023-02-09 00:10:00")

In [ ]:
signal_df

In [ ]:
#buy order
#sell order
#update_portfolio
#

start_time = start_period
end_time = end_period

for timestamp in pd.date_range(start=start_time, end=end_time, freq='1min'):
    print(timestamp)
    for minute_signal in signal_df[signal_df.time==timestamp]:
        print(minute_signal)
        
        
    

In [ ]:
def mini_expand4(pair="GMT/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=1,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,buy_function=buy_min_up):
    print(f"mini_expand : {pair}")
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='1 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='5 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list15m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='15 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1h["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 H'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1d["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 d'):Pair_Full.iloc[len(Pair_Full)-1].name],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    # Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
    #         right_index=True, suffixes=('', ''))
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='inner',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    #buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    buy_function(Merged,buy_pourcent=buy_pourcent,sell_pourcent=sell_pourcent,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    # Merged=justlast_remover(Merged)
    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    Merged=Merged.dropna()
    return Merged


In [ ]:
pair=coin
#df=full_expand(df_list1m[pair].iloc[loc_start:loc_end],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
df=mini_expand4(pair=coin,i=loc_start,j=loc_end,window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=TAKE_PROFIT,sell_pourcent=STOP_LOSS,buy_function=buy_test)


In [ ]:
# PAIR_WITH="BUSD"
# coins_to_download='LINK AGIX SAND JASMY T'
# # for coin in VOLATILE_COINS:
# #     coins_to_download=coins_to_download+" "+coin
# os.system(f"node database/ddargs.js {coins_to_download} {PAIR_WITH}")#node database/ddargs.js ORN BUSD

In [ ]:
# Mini_list=coins_to_download.split(" ")
# Mini_list=for c in Mini_list c=c+"/BUSD"

In [ ]:
import ccxt
import pandas as pd

# Load historical data for each coin
exchange = ccxt.binance()
symbols = ['BTC/USDT', 'ETH/USDT', 'LTC/USDT']
timeframe = '1d'
dataframes = []
for symbol in symbols:
    df = pd.DataFrame(exchange.fetch_ohlcv(symbol, timeframe))
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df['symbol'] = symbol
    dataframes.append(df)

# Merge dataframes for all coins into a single dataframe
df = pd.concat(dataframes)

# Clean and preprocess the data
df = df.drop_duplicates(subset=['timestamp', 'symbol'])
df = df.pivot(index='timestamp', columns='symbol', values='close')

# Create signal dataframe
signal_df = pd.DataFrame(index=df.index, columns=df.columns, data=0)
# Set signals based on your strategy
signal_df.loc[df['BTC/USDT'] > df['ETH/USDT'], 'BTC/USDT'] = 1
signal_df.loc[df['ETH/USDT'] > df['LTC/USDT'], 'ETH/USDT'] = 1
signal_df.loc[df['LTC/USDT'] > df['BTC/USDT'], 'LTC/USDT'] = 1

# Calculate coin returns based on signals
returns_df = df.pct_change() * signal_df.shift(1)

# Calculate portfolio return
weights = [0.5, 0.3, 0.2]  # weights for each coin in portfolio
portfolio_return = returns_df.dot(weights)

# Calculate total return over backtest period
total_return = (portfolio_return + 1).cumprod()[-1] - 1

# Evaluate strategy performance
benchmark_return = df['BTC/USDT'].pct_change

In [ ]:
df=df_list1m["LTC/USDT"].copy()

In [ ]:
def buy_optimal(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE):
    # df = df.fillna(0)
    max_forecast_size=window#MAX_FORCAST_SIZE
    after_dip_val=3
    try:
        print(f"optimalbuy buy maximum forcast size={max_forecast_size} at {buy_pourcent}% of the current price ")
    except:
        max_forecast_size = 3
        print("optimalbuy buy default window=3")
        
    mino = buy_pourcent / 100.0
    maxo = sell_pourcent / 100.0
    
    rolling_max_close_diff = ((df['close'].rolling(window=window).max().shift(-window+1) / df['close']) - 1).fillna(0)
    df['buy']=(rolling_max_close_diff >= mino).astype(int)
    
    # Compute rolling minimum values
    
    window_list=[7,window]#[3, 5, 7, 10, 15, 20]
    
    for window_size in window_list:
        col_name = f'ismin{window_size}'
        rolling_min = (df['close'].shift(after_dip_val) <= df.shift(-window_size-1)['close'].rolling(2*window_size).min())
        df = df.assign(**{col_name: rolling_min.astype(int)})

    df['ismin'] = df[[f'ismin{window_size}' for window_size in window_list ]].any(axis=1).astype(int)        

    # # Compute buy and sell signals
    rolling_low_close_diff =  ((df['low'].rolling(window=int(window/2)).min().shift(-int(window/2)+1)/ df['close'] ) -1).fillna(0)
    df['sell'] = (rolling_low_close_diff <= -maxo).astype(int)

    
    # Compute final buy signal
    df['buy'] = ((df['buy'] == 1) & (df['sell'] == 0) & (df['ismin'] == 1)).astype(int)
    # Remove unnecessary columns
    df = df.drop(columns=['sell', 'ismin'] + [f'ismin{window_size}' for window_size in window_list], errors='ignore')
    return df

In [ ]:
df=buy_optimal(df,buy_pourcent=0.5,sell_pourcent=0.3,window=20)

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5.5)

x = np.linspace(0, 10, 500)
dashes = []#[10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off

fig, ax = plt.subplots()
line1, = ax.plot(df.index[i_start:i_end], df.close[i_start:i_end], '-', linewidth=1,
                 label='price',c="w")
line1.set_dashes(dashes)

plt.plot(df[i_start:i_end][df.buy[i_start:i_end]==1].index, df[i_start:i_end][df.buy[i_start:i_end]==1].close, 'ro',c='g')

ax.legend(loc='lower right')
plt.show()



# --4 Initial Test

In [ ]:
## Generate Data
BAD_PERIOD_START="2022-08-30"
BAD_PERIOD_END="2023-01-22"
pair_to_test="ETC/USDT"
MAX_FORCAST_SIZE=20

# ai_options
WINDOW_SIZE= 14

start_period = pd.Timestamp(PERIODE_START)#datetime.strptime(PERIODE_START, '%d-%m-%Y %H:%M:%S')
end_period = pd.Timestamp(PERIOD_END)


#trading_options:
TAKE_PROFIT= 0.4
STOP_LOSS=  3.3  
MAX_HOLDING_TIME= MAX_FORCAST_SIZE

USE_TRAILING_STOP_LOSS: False
TRAILING_STOP_LOSS= .002     
TRAILING_TAKE_PROFIT= .002  





loc_start=100
loc_end=100000


i_start=0
i_end=i_start+1440

loc_start=df_list1m[pair_to_test].index.get_loc(start_period) #df_list1m[pair_to_test].index.get_loc(pd.to_datetime(BAD_PERIOD_START))
loc_end=df_list1m[pair_to_test].index.get_loc(end_period)


OnePair_DF=mini_expand4(        pair=pair_to_test,
                                i=loc_start,j=loc_end,
                                window=WINDOW_SIZE,
                                metadata=MetaData,
                                high_weight=1,
                                buy_pourcent=TAKE_PROFIT,
                                sell_pourcent=STOP_LOSS,
                                buy_function=buy_test2#buy_minimum#buy_fix#buy_test
                        )
OnePair_DT=OnePair_DF.to_numpy()
gc.collect()
OnePair_DT=fixdt(OnePair_DT)
print(OnePair_DT[0,0] == OnePair_DF.iloc[0,0])
print(OnePair_DT[5,5] == OnePair_DF.iloc[5,5])
hp(OnePair_DF.buy.mean(),"Buy mean percent")


## Ploting original
plot_data("Original Model", pair_to_test, 100, OnePair_DF, i_start, 700, OnePair_DF.buy,dot_color="r")


In [ ]:
# # df_list1m["LTC/USDT"].pop("buy0")
# # df_list1m["LTC/USDT"].pop("buy1")
# # df_list1m["LTC/USDT"].pop("sell0")

# OnePair_DF.pop("buy0")
# OnePair_DF.pop("buy1")
# OnePair_DF.pop("sell0")


In [ ]:
# Model_FileName="/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re3.h5" #LTC ::80.99/0.86 # GMT: 84/0.93 LPT:86.9330453563715/0.8 ETH:78 #SOL 75/0.6 Doge:58/0.5


# # Model_FileName= "/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp349_w12_max12min_Model_VeryDeep.h5" # Win Ratio:47.11246200607903 # GMT  Win Ratio:57.608695652173914
# #Model_FileName= "/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp250_w12_max12min_Model_VeryDeep.h5" # Win Ratio:42.80420927819772 #GMT Win Ratio:53.
# #Model_FileName="/UltimeTradingBot/Data/BUY_TEST/tp180_w12_max12min_Model_VeryDeep.h5" # Win Ratio:59.35050391937291 # GMT 61.70886075949367/9 #SNM 52% / ETH 59.8/2
# #Model_FileName="/UltimeTradingBot/Data/BUY_TEST/tp70_w12_max12min_Model_VeryDeep.h5" # %44
# # Model_FileName="/UltimeTradingBot/Data/BUY_TEST/tp300_w12_max60min_Model_true_win_model_v2.h5" #45%
# #Model_FileName="/UltimeTradingBot/Data/BUY_TEST/tp300_w12_max60min_Model_VeryDeep.h5" #46%

#Model_FileName="/UltimeTradingBot/Data/BUY_OPTIMAL/tp40_w14_max7min_Anti-Model_v2.h5"
Model_FileName="/UltimeTradingBot/Data/BUY_OPTIMAL/tp40_w14_max7min_Model_VeryDeep.h5"
USED_MODEL=load_model(Model_FileName)




plot_data("Original Model", pair_to_test, 100, OnePair_DF, i_start, 700, OnePair_DF.buy,dot_color="r")





OnePair_PredNote=USED_MODEL.predict( OnePair_DT[:, 0:-1])
OnePair_Pred=OnePair_PredNote.round()

gc.collect()

Original_Traget_Data=OnePair_DT[:,-1]
buy_mean=hp(Original_Traget_Data.mean(),"Buy Mean: ")
Predicted_Data=OnePair_Pred[:,0]
gc.collect()
TruePred=(Original_Traget_Data==Predicted_Data).copy()
ModelAccuracy=hp(TruePred.mean(),"ModelAccuracy")
gc.collect()
TrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
TrueWinPred_Mean=hp(TrueWinPred.mean(),"True Win Predictions Mean of all")
gc.collect()
LossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
LossPred_Mean=hp(LossPred.mean(),"XXX Loss Buy Mean of all")
gc.collect()

MissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
MissedDeal_Mean=hp(MissedDealPred.mean(),"Missed good deal off all")
gc.collect()

GoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
GoodZero_Mean=hp(GoodZeroPred.mean(),"Good Zero prediction Mean")
gc.collect()

fiability=TrueWinPred_Mean + LossPred_Mean + MissedDeal_Mean + GoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")
winratio=TrueWinPred_Mean/(LossPred_Mean+TrueWinPred_Mean)

print(f"========= Win Ratio:{winratio*100} %====================")
TestsHistory.loc[len(TestsHistory)]=[Model_FileName,pair_to_test ,ModelAccuracy, TrueWinPred_Mean, LossPred_Mean, MissedDeal_Mean, GoodZero_Mean, WINDOW_SIZE,MAX_FORCAST_SIZE,TAKE_PROFIT,hp(winratio),buy_mean]


PREDICTION_TO_TEST=Predicted_Data
plot_data(Model_FileName, pair_to_test, winratio, OnePair_DF, i_start, 700, PREDICTION_TO_TEST)

gc.collect()

In [ ]:
TestsHistory= pd.DataFrame(columns= ['Model-FileName','TestedPair' ,'ModelAccuracy', 'TrueWin', 'XLoss', 'MissedDeal', 'GoodZero', 'WINDOW','MAX_FORCAST_SIZE','TAKE_PROFIT','WinRatio','BuyMean'])

In [ ]:
## Generate Data
BAD_PERIOD_START="2022-08-30"
BAD_PERIOD_END="2022-11-22"
MAX_FORCAST_SIZE=16

# ai_options
WINDOW_SIZE= 7

start_period = datetime.strptime(PERIODE_START, '%d-%m-%Y %H:%M:%S')
end_period = datetime.strptime(PERIODE_END, '%d-%m-%Y %H:%M:%S')


#trading_options:
TAKE_PROFIT= 0.4
STOP_LOSS=  1.3  
MAX_HOLDING_TIME= MAX_FORCAST_SIZE

USE_TRAILING_STOP_LOSS: False
TRAILING_STOP_LOSS= .002     
TRAILING_TAKE_PROFIT= .002  





loc_start=0
loc_end=1000000


i_start=0
i_end=i_start+1000

# loc_start=df_list1m[pair_to_test].index.get_loc(pd.to_datetime(BAD_PERIOD_START))
# loc_end=df_list1m[pair_to_test].index.get_loc(pd.to_datetime(BAD_PERIOD_END))


Model_FileNames=["/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re3.h5" #LTC ::80.99/0.86 # GMT: 84/0.93 LPT:86.9330453563715/0.8 ETH:78 #SOL 75/0.6 Doge:58/0.5
,"/UltimeTradingBot/Data/BUY_TEST/tp30_w7_max2min_Model_VeryDeep.h5"   #52.500060887990465/21 ETC:51.307/16
,"/UltimeTradingBot/Data/BUY_TEST/tp30_w7_max2min_Model_true_win_model_Re1.h5"   #GMT 65.888/2% -  ETh: 55.0/0.37 -LTC 54/1.8
,"/UltimeTradingBot/Data/BUY_TEST/tp30_w7_max2min_Model_true_win_model_Re2.h5"
,"/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re2.h5" #LTC: 64/4
,"/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re1.h5" #LTC: 57.15/7.9
,"/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_VeryDeep.h5" #LTC:48.741696716644526/12
,"/UltimeTradingBot/Data/BUY_MIN_CLOSE/tp70_w7_max4min_Model_vInit.h5" #
,"/UltimeTradingBot/Data/BUY_MIN_CLOSE/tp70_w7_max4min_Model_true_win_model_Re1.h5" #Doge:28/4.5
,"/UltimeTradingBot/Data/BUY_MIN_CLOSE/tp70_w7_max4min_Model_true_win_model_Re2.h5" #Doge:32/4.0
,"/UltimeTradingBot/Data/BUY_MIN_CLOSE/tp70_w7_max4min_Model_true_win_model_Re3.h5" #Doge:34.17/3.8 GMT35
,"/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re2.h5" #LTC ::80.99/0.86 # GMT: 84/0.93 LPT:86.9330453563715/0.8 ETH:78 #SOL 75/0.6 Doge:58/0.5
,"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp42_w7_max16min_Model_true_win_model_Re3.h5" #GMT 44/
,"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp42_w7_max16min_Model_JUSTGood_v2.h5"
,"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp42_w7_max16min_Model_JUSTGood_v2.h5" # GMT:58/8
,"/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re2.h5" #58/6
,"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp130_w7_max16min_Model_true_win_model_Re7.h5",
"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp130_w7_max16min_Anti-Model_v2.h5"

]

Model_FileNames=["/UltimeTradingBot/Data/BUY_TEST/tp92_w7_max3min_Model_true_win_model_Re4.h5",
                 "/UltimeTradingBot/Data/BUY_TEST/tp92_w7_max3min_Model_true_win_model_Re4.h5",
                 "/UltimeTradingBot/Data/BUY_TEST/tp92_w7_max3min_Model_VeryDeep.h5",
                 "/UltimeTradingBot/Data/BUY_TEST/tp92_w7_max3min_Model_vInit.h5"]

Model_FileNames=["/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re3.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re4.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re2.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re1.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_VeryDeep.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_vInit.h5"
    
]


Model_FileNames=matching_files
for pair_to_test in ["GMT/USDT","XRP/USDT","ETH/USDT","LTC/USDT","SOL/USDT"]:
    coin=pair_to_test
    loc_start=df_list1m[coin].index.get_loc(day)
    loc_end=df_list1m[coin].index.get_loc(day+pd.Timedelta('1 day'))
    print(f'#################################################### Working on : {pair_to_test} ###################################################')
    OnePair_DF=mini_expand4(        pair=pair_to_test,
                                    i=loc_start,j=loc_end,
                                    window=WINDOW_SIZE,
                                    metadata=MetaData,
                                    high_weight=1,
                                    buy_pourcent=TAKE_PROFIT,
                                    sell_pourcent=STOP_LOSS,
                                    buy_function=buy_test
                            )
    OnePair_DT=OnePair_DF.to_numpy()
    gc.collect()
    OnePair_DT=fixdt(OnePair_DT)
    print(OnePair_DT[0,0] == OnePair_DF.iloc[0,0])
    print(OnePair_DT[5,5] == OnePair_DF.iloc[5,5])
    hp(OnePair_DF.buy.mean(),"Buy mean percent")
    x = np.linspace(0, 10, 500)
    dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off

    fig, ax = plt.subplots()
    line1, = ax.plot(OnePair_DF.index[i_start:i_end], OnePair_DF.price[i_start:i_end], '-', linewidth=1,
                    label='Dashes set retroactively')
    line1.set_dashes(dashes)
    plt.plot(OnePair_DF[i_start:i_end][OnePair_DF.buy[i_start:i_end]==1].index, OnePair_DF[i_start:i_end][OnePair_DF.buy[i_start:i_end]==1].price, 'ro')


    ax.legend(loc='lower right')
    plt.show()


    for Model_FileName in Model_FileNames:
        print(f'Using Model:{Model_FileName}')
        USED_MODEL=load_model(Model_FileName)

  


        OnePair_PredNote=USED_MODEL.predict( OnePair_DT[:, 0:-1])
        OnePair_Pred=OnePair_PredNote.round()

        gc.collect()

        Original_Traget_Data=OnePair_DT[:,-1]
        buy_mean=hp(Original_Traget_Data.mean(),"Buy Mean: ")
        Predicted_Data=OnePair_Pred[:,0]
        gc.collect()
        TruePred=(Original_Traget_Data==Predicted_Data).copy()
        ModelAccuracy=hp(TruePred.mean(),"ModelAccuracy")
        gc.collect()
        TrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
        TrueWinPred_Mean=hp(TrueWinPred.mean(),"True Win Predictions Mean of all")
        gc.collect()
        LossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
        LossPred_Mean=hp(LossPred.mean(),"XXX Loss Buy Mean of all")
        gc.collect()

        MissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
        MissedDeal_Mean=hp(MissedDealPred.mean(),"Missed good deal off all")
        gc.collect()

        GoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
        GoodZero_Mean=hp(GoodZeroPred.mean(),"Good Zero prediction Mean")
        gc.collect()

        fiability=TrueWinPred_Mean + LossPred_Mean + MissedDeal_Mean + GoodZero_Mean
        if( fiability == 100):print("good fiability")
        else: print(f"check the fiability {fiability}")
        winratio=TrueWinPred_Mean/(LossPred_Mean+TrueWinPred_Mean)

        print(f"========= Win Ratio:{winratio*100} %====================")
        TestsHistory.loc[len(TestsHistory)]=[Model_FileName,pair_to_test ,ModelAccuracy, TrueWinPred_Mean, LossPred_Mean, MissedDeal_Mean, GoodZero_Mean, WINDOW_SIZE,MAX_FORCAST_SIZE,TAKE_PROFIT,hp(winratio),buy_mean]
        TestsHistory.to_csv("Plots/mini-results.csv")

        PREDICTION_TO_TEST=Predicted_Data

        x = np.linspace(0, 10, 500)
        dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off
        fig, ax = plt.subplots()
        line1, = ax.plot(OnePair_DF.index[i_start:i_end], OnePair_DF.price[i_start:i_end], '-', linewidth=1,
                        label='price')
        line1.set_dashes(dashes)
        plt.plot(OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].index, OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].price, 'ro')
        mname=Model_FileName.replace("/UltimeTradingBot/Data","")
        mtitle=f"{coin} - {mname}".replace("/","-")
        plt.title(mtitle)
        plt.savefig(f"Plots/{mtitle}.png")        
        plt.show()
        gc.collect()

In [ ]:
TestsHistory2=TestsHistory

In [ ]:
TestsHistory=pd.read_csv("test-restuls01.csv",index_col=0)

In [ ]:
# TestsHistory.sort_values("WinRatio",ascending=0)[:30]
TestsHistory.sort_values("TestedPair",ascending=0)[:30]

In [ ]:
TestsHistory.sort_values("TrueWin",ascending=0)[:20]

In [ ]:
TestsHistory.groupby("Model-FileName").mean().sort_values("WinRatio",ascending=0)[:10]

In [ ]:
TestsHistory.groupby("Model-FileName").mean().sort_values("WinRatio",ascending=0)[:10]

# Test MultiModel sollution

In [ ]:

bag_of_modules=["/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp130_w7_max16min_Anti-Model_v2.h5",
               "/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp130_w7_max16min_Model_VeryDeep.h5",
               "/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp42_w7_max16min_Model_vInit.h5",
               "/UltimeTradingBot/Data/SELL_TEST/tp70_w7_max3min_Model_vInit.h5",
               #"/UltimeTradingBot/Data/BUY_TEST/tp150_w7_max2min_Model_true_win_model_v3.h5", 
               #"/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re1.h5"
               ]

bag_of_modules=[
                "/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp130_w7_max16min_Model_vInit.h5",
                #"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp130_w7_max16min_Model_true_win_model_Re8.h5",
                #"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp130_w7_max16min_Anti-Model_v2.h5",
                #"/UltimeTradingBot/Data/AFTER_DEPTH_CLOSE/tp42_w7_max16min_Model_vInit.h5",
                "/UltimeTradingBot/Data/BUY_MIN_CLOSE/tp70_w7_max4min_Model_VeryDeep.h5",
                "/UltimeTradingBot/Data/BUY_TEST/tp150_w7_max2min_Model_vInit.h5",  
 #               "/UltimeTradingBot/Data/SELL_TEST/tp250_w7_max16min_Model_vInit.h5", 
 #               "/UltimeTradingBot/Data/SELL_TEST/tp25_w7_max1min_Anti-Model_v2.h5",               
                ]

bag_of_modules=[
                # "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re3.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re4.h5",
                #  "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re2.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re1.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_VeryDeep.h5",
                 "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_vInit.h5"
    
]

aditive_models=["/UltimeTradingBot/Data/SELL_TEST/tp25_w7_max1min_Model_vInit.h5",
                "/UltimeTradingBot/Data/SELL_TEST/tp25_w7_max1min_Anti-Model_v2.h5",
                "/UltimeTradingBot/Data/SELL_TEST/tp40_w7_max3min_Anti-Model_v2.h5",
                "/UltimeTradingBot/Data/SELL_TEST/tp40_w7_max3min_Model_BadVeryDeep_v2.h5",
                "/UltimeTradingBot/Data/SELL_TEST/tp250_w7_max16min_Model_VeryDeep.h5",
                "/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_v5.h5",  
                "/UltimeTradingBot/Data/BUY_TEST/tp70_w7_max2min_Model_true_win_model_Re3.h5",
                "/UltimeTradingBot/Data/SELL_TEST/tp70_w7_max3min_Model_vInit.h5",
                
                              
                ]
aditive_models=[]
import matplotlib.pyplot as plt

x = np.linspace(0, 10, 500)
dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off

fig, ax = plt.subplots()
line1, = ax.plot(OnePair_DF.index[i_start:i_end], OnePair_DF.price[i_start:i_end], '-', linewidth=1,
                 label='Dashes set retroactively')
line1.set_dashes(dashes)
plt.plot(OnePair_DF[i_start:i_end][OnePair_DF.buy[i_start:i_end]==1].index, OnePair_DF[i_start:i_end][OnePair_DF.buy[i_start:i_end]==1].price, 'ro')


ax.legend(loc='lower right')
plt.show()

OnePair_Pred=1
if aditive_models:
    for Model_FileName in aditive_models:
        USED_MODEL=load_model(Model_FileName)
        OnePair_PredNote=USED_MODEL.predict( OnePair_DT[:, 0:-1])
        OnePair_Pred=(OnePair_PredNote.round()==1)|(OnePair_Pred==1)
if bag_of_modules:
    for Model_FileName in bag_of_modules:
        USED_MODEL=load_model(Model_FileName)
        OnePair_PredNote=USED_MODEL.predict( OnePair_DT[:, 0:-1])
        OnePair_Pred=OnePair_PredNote.round()*OnePair_Pred

gc.collect()

Original_Traget_Data=OnePair_DT[:,-1]
buy_mean=hp(Original_Traget_Data.mean(),"Buy Mean: ")
Predicted_Data=OnePair_Pred[:,0]
gc.collect()
TruePred=(Original_Traget_Data==Predicted_Data).copy()
ModelAccuracy=hp(TruePred.mean(),"ModelAccuracy")
gc.collect()
TrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
TrueWinPred_Mean=hp(TrueWinPred.mean(),"True Win Predictions Mean of all")
gc.collect()
LossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
LossPred_Mean=hp(LossPred.mean(),"XXX Loss Buy Mean of all")
gc.collect()

MissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
MissedDeal_Mean=hp(MissedDealPred.mean(),"Missed good deal off all")
gc.collect()

GoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
GoodZero_Mean=hp(GoodZeroPred.mean(),"Good Zero prediction Mean")
gc.collect()

fiability=TrueWinPred_Mean + LossPred_Mean + MissedDeal_Mean + GoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")
winratio=TrueWinPred_Mean/(LossPred_Mean+TrueWinPred_Mean)

print(f"========= Win Ratio:{winratio*100} %====================")
TestsHistory.loc[len(TestsHistory)]=[Model_FileName,pair_to_test ,ModelAccuracy, TrueWinPred_Mean, LossPred_Mean, MissedDeal_Mean, GoodZero_Mean, WINDOW_SIZE,MAX_FORCAST_SIZE,TAKE_PROFIT,hp(winratio),buy_mean]


PREDICTION_TO_TEST=Predicted_Data

x = np.linspace(0, 10, 500)
dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off
fig, ax = plt.subplots()
line1, = ax.plot(OnePair_DF.index[i_start:i_end], OnePair_DF.price[i_start:i_end], '-', linewidth=1,
                 label='price')
line1.set_dashes(dashes)
plt.plot(OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].index, OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].price, 'ro')


ax.legend(loc='lower right')
plt.show()
gc.collect()

In [ ]:
ii_start=49000
ii_end=ii_start+1000
x = np.linspace(0, 10, 500)
dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off
fig, ax = plt.subplots()
line1, = ax.plot(OnePair_DF.index[ii_start:ii_end], OnePair_DF.price[ii_start:ii_end], '-', linewidth=1,
                 label='price')
line1.set_dashes(dashes)
plt.plot(OnePair_DF[ii_start:ii_end][PREDICTION_TO_TEST[ii_start:ii_end]==1].index, OnePair_DF[ii_start:ii_end][PREDICTION_TO_TEST[ii_start:ii_end]==1].price, 'ro')


ax.legend(loc='lower right')
plt.show()
gc.collect()

In [ ]:
ii_start=46000
ii_end=ii_start+600
x = np.linspace(0, 10, 500)
dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off
fig, ax = plt.subplots()
line1, = ax.plot(OnePair_DF.index[ii_start:ii_end], OnePair_DF.price[ii_start:ii_end], '-', linewidth=1,
                 label='price')
line1.set_dashes(dashes)
plt.plot(OnePair_DF[ii_start:ii_end][PREDICTION_TO_TEST[ii_start:ii_end]==1].index, OnePair_DF[ii_start:ii_end][PREDICTION_TO_TEST[ii_start:ii_end]==1].price, 'ro')


# ax.legend(loc='lower right')

plt.show()
gc.collect()

In [ ]:
!ls Plots

In [ ]:
press=-0.37
OnePair_Pred=(OnePair_PredNote+press).round()
gc.collect()

Original_Traget_Data=OnePair_DT[:,-1]
buy_mean=hp(Original_Traget_Data.mean(),"Buy Mean: ")
Predicted_Data=OnePair_Pred[:,0]
gc.collect()
TruePred=(Original_Traget_Data==Predicted_Data).copy()
ModelAccuracy=hp(TruePred.mean(),"ModelAccuracy")
gc.collect()
TrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
TrueWinPred_Mean=hp(TrueWinPred.mean(),"True Win Predictions Mean of all")
gc.collect()
LossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
LossPred_Mean=hp(LossPred.mean(),"XXX Loss Buy Mean of all")
gc.collect()

MissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
MissedDeal_Mean=hp(MissedDealPred.mean(),"Missed good deal off all")
gc.collect()

GoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
GoodZero_Mean=hp(GoodZeroPred.mean(),"Good Zero prediction Mean")
gc.collect()

fiability=TrueWinPred_Mean + LossPred_Mean + MissedDeal_Mean + GoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")
winratio=TrueWinPred_Mean/(LossPred_Mean+TrueWinPred_Mean)

print(f"========= Win Ratio:{winratio*100} %====================")
TestsHistory.loc[len(TestsHistory)]=[Model_FileName,pair_to_test ,ModelAccuracy, TrueWinPred_Mean, LossPred_Mean, MissedDeal_Mean, GoodZero_Mean, WINDOW_SIZE,MAX_FORCAST_SIZE,TAKE_PROFIT,hp(winratio),buy_mean]


PREDICTION_TO_TEST=Predicted_Data

x = np.linspace(0, 10, 500)
dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off
fig, ax = plt.subplots()
line1, = ax.plot(OnePair_DF.index[i_start:i_end], OnePair_DF.price[i_start:i_end], '-', linewidth=1,
                 label='price')
line1.set_dashes(dashes)
plt.plot(OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].index, OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].price, 'ro')


ax.legend(loc='lower right')
plt.show()
gc.collect()

In [ ]:
TestsHistory[TestsHistory.WinRatio==78.558]["Model-FileName"].values

In [ ]:
TestsHistory[TestsHistory.WinRatio==60.909]

In [ ]:
TestsHistory.to_csv("test-restuls02.csv")

In [ ]:
import os
import glob

path = "/UltimeTradingBot/Data"
pattern = os.path.join(path, "**/*w7*.h5")

matching_files = []
for file_path in glob.iglob(pattern, recursive=True):
    if os.path.isfile(file_path):
        matching_files.append(file_path)

print(matching_files)